<a href="https://colab.research.google.com/github/krishnanlavanya/Proso-Project-Dataset/blob/main/ResumeSkillProfileMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re


In [ ]:
df1 = pd.read_csv('/content/dataF.csv')
print(df1.head())  # Display the first few rows of the dataset


   Unnamed: 0                          company  \
0           1          Visual BI Solutions Inc   
1           2                       Jobvertise   
2           3           Santander Consumer USA   
3           4   Federal Reserve Bank of Dallas   
4           5                           Aviall   

                                            position  \
0  Graduate Intern (Summer 2017) - SAP BI / Big D...   
1                          Digital Marketing Manager   
2    Manager, Pricing Management Information Systems   
3               Treasury Services Analyst Internship   
4                              Intern, Sales Analyst   

                                                 url     location  \
0  https://www.glassdoor.com/partner/jobListing.h...    Plano, TX   
1  https://www.glassdoor.com/partner/jobListing.h...   Dallas, TX   
2  https://www.glassdoor.com/partner/jobListing.h...   Dallas, TX   
3  https://www.glassdoor.com/partner/jobListing.h...   Dallas, TX   
4  https://www.gl

In [ ]:
df2 = pd.read_csv('/content/resumeskills.csv')
print(df2.head())

        Position                                             Resume  \
0   Data Science  Skills * Programming Languages: Python (pandas...   
1             HR  I.T. Skills â¢ Windows XP, Ms Office (Word, E...   
2       advocate  Skills Legal Writing Efficient researcher Lega...   
3           Arts  â¢ Operating Systems: Windows XP / Vista / 07...   
4  WEB DESIGNING  Technical Skills Web Technologies: Angular JS,...   

                                              Skills Unnamed: 3  Unnamed: 4  \
0                         JAVA SCRIPT, JQUERY,PYTHON        NaN         NaN   
1                                      HR MANAGEMENT        NaN         NaN   
2                          legal writing, litigation        NaN         NaN   
3                          MS CIT, WINDOWS XP, VISTA        NaN         NaN   
4  PHOTOSTRAP. BOOTSTRAP, HTML5, CSS3, JAVASCRIPT...        NaN         NaN   

   Unnamed: 5  Unnamed: 6  
0         NaN         NaN  
1         NaN         NaN  
2         NaN 

In [ ]:
def extract_skills(description):
    pattern = r'\b([A-Z][a-zA-Z]+)\b'
    skills = re.findall(pattern, description)
    return ', '.join(skills)

df1['Skills'] = df1['Job Description'].apply(extract_skills)

print(df1.head())

   Unnamed: 0                          company  \
0           1          Visual BI Solutions Inc   
1           2                       Jobvertise   
2           3           Santander Consumer USA   
3           4   Federal Reserve Bank of Dallas   
4           5                           Aviall   

                                            position  \
0  Graduate Intern (Summer 2017) - SAP BI / Big D...   
1                          Digital Marketing Manager   
2    Manager, Pricing Management Information Systems   
3               Treasury Services Analyst Internship   
4                              Intern, Sales Analyst   

                                                 url     location  \
0  https://www.glassdoor.com/partner/jobListing.h...    Plano, TX   
1  https://www.glassdoor.com/partner/jobListing.h...   Dallas, TX   
2  https://www.glassdoor.com/partner/jobListing.h...   Dallas, TX   
3  https://www.glassdoor.com/partner/jobListing.h...   Dallas, TX   
4  https://www.gl

In [ ]:
def calculate_match_percentage(job_description, resume):
    vectorizer = CountVectorizer()
    job_desc_vector = vectorizer.fit_transform([job_description])
    resume_vector = vectorizer.transform([resume])
    cosine_sim = cosine_similarity(job_desc_vector, resume_vector)[0][0]
    match_percentage = cosine_sim * 100
    rounded_match_percentage = round(match_percentage, 2)
    return rounded_match_percentage

matched_rows = []

for index, row in df1.iterrows():
    skills = row['Skills'].split(', ')
    job_description = row['Job Description']
    matches = []
    for resume_index, row2 in df2.iterrows():
        resume_skills = row2['Skills']
        resume = row2['Resume']
        matching_skills = [skill for skill in skills if skill.lower() in resume_skills.lower()]
        if matching_skills:
            match_percentage = calculate_match_percentage(job_description, resume)
            matches.append((resume_index, match_percentage))
    if matches:
        matched_rows.extend([(index, resume_index, match_percentage) for resume_index, match_percentage in matches])

if len(matched_rows) > 0:
    matched_df = pd.DataFrame(matched_rows, columns=['Job Description Index', 'Resume Index', 'Rounded Final'])
    matched_df.to_csv('matched_data_CS.csv', index=False)
    print("Matched data exported to 'matched_data_CS.csv'")
else:
    print("No matches of resumes for any job description found.")


Matched data exported to 'matched_data_CS.csv'


In [ ]:
Final = pd.read_csv('matched_data_CS.csv')

print(Final)

Final.nlargest(5, ['Rounded Final'])


      Job Description Index  Resume Index  Rounded Final
0                         0             2          35.10
1                         0             3          40.27
2                         0             4          43.24
3                         0             5          40.45
4                         0             6          39.09
...                     ...           ...            ...
3725                    156            34          70.20
3726                    156            36          79.11
3727                    156            37          55.00
3728                    156            38          69.91
3729                    156            39          63.25

[3730 rows x 3 columns]


,Job Description Index,Resume Index,Rounded Final
2963,126,18,89.01
376,15,18,87.23
3237,137,18,86.86
2397,99,36,86.69
3424,144,36,86.69
